## Data Science Immesive Final Project (General Assembly ATX Spring 2017)
### Can Machine Learning Predict Time on Trail for Appalachian Trail Thru hikers?

### Final Project Question: Can you predict how long it will take an Appalachian Trail Thru Hiker to complete the Appalachian Trail based on the features of this dataset?


#### Relevant information
The data for my final project was obtained from the Appalachian Trail Conservancy (Harpers Ferry,WV).

For the purposes of this project. I will exclude flip flops (only accounted for 1 data point in the data set) and thru-section hikes. A thru hike for the purposes of this project will be defined as the self reporting of that a hiker has created a continious footpath from Georgia to Maine or Maine to Georgia in a single hiking season.


# Data Cleaning Table of Content: 
I explore the data column by column to understand what sort of missing or inaccurate values are contained.

def (clean_1): Columns that can/will be dropped
- Rename columns to be more accurate.
- Drop 'Days Elapsed' column as it does not accuratly calculate the number of days on trail. (I replace this column in clean_2 with 'Total_Days')
- Drop 'Hiking_Disability' Column as it was not well reported.
- Drop Dog Weight Column (16238 of possible 17835 values were 0)
- Drop columns with too many null values (columns that exceed 85 percent null values)
- drop null values out of columns
- **** come back to this Drop Under2kAmount column since (16125 out of 17835) values were 0 essentially the same as null values 
- Dropped 'Hiking 


def (clean_2/clean_3): Clean up the Age column 
- Fill null values in 'Age' column with median age.
- All values below 4(There were some negative values) 
- The youngest person to thru-hike the AT is 5 so I assume all values above this are 

 
    
def (clean_4): Datetime objects, 
-  Convert 'Start' and 'Finish' columns to Datetime objects
-  Create a new column 'Total Days' that calculates total days on trail (at['Finish] - at['Start'])) 
-  There was as single null value in completion year, however, the finish date showed the hiker completed in 2016 so I changed the value to 2016 
- I conveted the completion yr column from floats to ints 
- converted completion yr to a datetime object



def (clean_5/clean_6): Clean 'Sex' column
- In (clean_1) I dropped all null values in this column 
- Drop 'oher' and 'othe' out of the sex column
- Convert the 'Sex' column to numerical format with the map function
- Drop null values in the Sex column. There were only 2. 
- Converted M/F from float to int

def (clean_7/clean_8): Clean 'Hike_Type' column (Drop section, alternative hikes, Yo Yo)
- Drop values in the Hike column that are labeled alternative as I am unsure what these are and how to define them.
- Since I am only interested in Thru Hikers and trying to predict time on trail based on people trying to hike in 6 months, and since all thru section hikers take varying amounts of time. 
- Difficult to estimate value for a Yo Yo so I dropped it, it was only a single column

def (clean_9): Categorize the race column: Since the Data Set was very disorganized, Regular Expression
was unable to take care of the categorization so I had to manually manipulate the categories
- Caucasion variation = 0
- African American Variation = 1
- Asian Variation = 2
- Hispanic Variation = 3
- Native American = 4
- Mixed Race: =5
- European = 6
- Mixed Race  = 7
- European/Not American (You cannot assume they are white because they are European) = 8
- Other = 9
    

def(clean_10): Group country of origin to express if they are North American, European, Australian,etc...
- North American = 0 
- European = 1 
- Oceania = 2
- African = 3

def(clean_11): Convert Hike_Type values to numbers
- 'NOBO':0
- 'SOBO':1 
- 'Flip':2 

def (clean_12): Map Hike Types
- NOBO = 0
- SOBO = 1
- Flip = 2

def (clean_13): Convert Total_Days to an int


def (clean_14): Drop Start and Finish Columns because you cannot pass datetime objects into your model



def (clean_16): Reset Index 
- The index needed to be reset since I had dropped many rows containing null values 



# To do 

3.) Make Boolean columns into dummy variables 

7.) State 

8.) There are negative total days (str.replace values)




In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
at = pd.read_excel('BenHikersInfo.xlsx')

In [3]:
at.shape

(17835, 28)

In [4]:
at[(at['AgeYrFin'] == 24) & (at['StAbbrev'] == 'TX')].head()

,Finish,AgeYrFin,Race or Heritage,StAbbrev,Sex,Country,Hike Type,Start,Days Elapsed,Yr Comp,...,SchoolCredit,School,Minority,MinorityComments,Vegetarian,DehydrateOwnFood,Giardia,Lyme,Norovirus,Disease Other
4788,10/10/1998,24.0,Prefer not to say,TX,M,USA,NOBO,3/20/1998,205.0,1998.0,...,False,NaN,False,NaN,False,False,False,False,False,NaN
5069,8 /15/1999,24.0,Prefer not to say,TX,M,USA,NOBO,3/15/1999,154.0,1999.0,...,False,NaN,False,NaN,False,False,False,False,False,NaN
5470,11/11/1999,24.0,Prefer not to say,TX,M,USA,Flip,4/8/1999,218.0,1999.0,...,False,NaN,False,NaN,False,False,False,False,False,NaN
6135,9 /29/2000,24.0,Prefer not to say,TX,M,USA,NOBO,4/3/2000,180.0,2000.0,...,False,NaN,False,NaN,False,False,False,False,False,NaN
6494,10/1 /2001,24.0,Prefer not to say,TX,F,USA,NOBO,3/31/2001,185.0,2001.0,...,False,NaN,False,NaN,False,False,False,False,False,NaN


## Cleaning Dataset

In [5]:
def clean_1(df):
    df.drop(labels = df[['Disease Other','School','Hiking Disability','DogWeightLbs','Days Elapsed','MinorityComments','School',
                         'DogBreed','Disability','Under2KAmount']], axis = 1, inplace = True)
    df.rename(columns = {'AgeYrFin': 'Age','Race or Heritage': 'Race',
                        'StAbbrev':'State', 'Hiking Disability ': 'Hiking_Disability', 
                         'Hike Type': 'Hike_Type', 'Yr Comp':'completion_yr',
                         },inplace = True)
    df.dropna(subset= ['Start','Sex','Race','Country','State','Hike_Type'],inplace = True)

clean_1(at)  


def clean_2(df):
    df['Age'] = df['Age'].fillna(df['Age'].median())
clean_2(at)


def clean_3(x,median_age): 
    if x < 4:
        return median_age
    else:
        return x 
at['Age'] = at['Age'].apply(lambda x: clean_3(x, at['Age'].median()))

In [6]:
def clean_4(df):
    df['Finish'] = pd.to_datetime(df['Finish'])
    df['Start'] = pd.to_datetime(df['Start'])
    df['Total_Days'] = df['Finish'] - df['Start']
    df['completion_yr'].fillna(2016,inplace = True)
    df['completion_yr'] = df['completion_yr'].astype(int)
clean_4(at)

In [7]:
def clean_5(df):
    #df['Sex'] = df['Sex'].map({'M':0,'F':1})
    to_drop = ['othe','Oher']
    x = df[df['Sex'].isin(to_drop)]
    y = x.index
    df.drop(y,inplace = True)
clean_5(at)

# def clean_6(df):
#     df.dropna(subset = ['Sex'], inplace = True)
#     df['Sex'] = df['Sex'].astype(int)
# clean_6(at)
    

In [8]:
def clean_7(df):
    to_drop = ['Alternative','alternative','Alterntive', 'Yo Yo']
    x = df[df['Hike_Type'].isin(to_drop)]
    y = x.index
    df.drop(y,inplace = True)
clean_7(at)

def clean_8(df):
    to_drop = ['Section']
    x = df[df['Hike_Type'].isin(to_drop)]
    y = x.index
    df.drop(y,inplace = True)
clean_8(at)

In [9]:
at['Sex'].value_counts()

M    9977
F    2964
Name: Sex, dtype: int64

In [10]:
# Recall to maintain original column 
def clean_9(df):
    
    # Caucasion Variation: 0 Identify Race Over Nationality/Religion 
    df['Race'] = df['Race'].replace(['White','WASP','Hewbrew/White','English','Caucasian / Viking',
                                    'Caucasian or White','Caucasian or White/Irish','Caucasian / Polish / Irish',
                                    'Caucasian / South Africaan','Caucasian or White/Amish','Caucasian / Irish'
                                     'Caucasian or White/Jewish','Caucasian / Irish',
                                     'Anglo Saxon / Scottish / American','Caucasian or White (Jewish)',
                                     'Caucasian / European','Caucasian or White/Pacific Islander','Jewish/White',
                                     'Caucasian or White/Jewish','Caucasian / non-Hispanic',
                                     'Caucasian / Czech / Russian / German / English)','Anglo Saxon','White/Irish',
                                     'White/Jewish','Caucasian/Lithuanian-American','White/Scandinavian','Caucasian',
                                     'White/Pennsylvania Dutch','Caucasian / Irish / Nordic','Caucasian / Jewish',
                                     'European/American', 'Caucasian / Polish','Caucasian / German',
                                     'Caucasian / 1/4 German', 'Caucasian/French Canandien','Caucasian / Slovak','Mennowhite',
                                     'White/Lebanese', 'White - Irish, Scot and Spanish', 'Caucasian / Italian / Eastern European',
                                     'Caucasian / Bahamian', 'Caucasian / Haitian'], 'Caucasian')
    # African American Variation: 1
    df['Race'] = df['Race'].replace(['African American','African American/Biracial',], 'African_American')
    
    # Asian Variation: 2
    df['Race'] = df['Race'].replace(['Asian','Chinese','Asian / Chinese','Vietnamese','Thai',
                                    'Taiwanese / American','Asian / American','Korean/American','Japanese',
                                    'Indian','Asian American','Korean / American','Asian (Indian/American)',
                                    'Middle Eastern','Korean','Japanese/American','Caucasian / Asian / Pacific Islander',
                                     'American Indian', 'Vietnamese American','Persian','Japanese / American', 'Pakistani',
                                     'Tamil/Sri Lankan', 'Pacific Islander', 'Caucasian / Asian'], 'Asian')
    # Hispanic/Latino: 3 I prioritize race over nationality 
    df['Race'] = df['Race'].replace(['Hispanic','Latino / Venzuelan','Japanese-American','Mexican / American',
                                    'Mexican','Hispanic  / Colombian','Cuban','Latino','Puerto Rican',
                                     'Hispanic or Latino','Mexican/American','Dominican','Hispanic / Mexican',
                                    'Brazilian','Hispanic / Mexican / American','Hispanic/German', 'Hispanic / Italian',
                                    ], 'Hispanic')
    # Native American Variation: 4
    df['Race'] = df['Race'].replace(['American Indian / Cherokee','Cherokee & English','Native American',
                                    'Native American (1/2 cherokee) / 1/2 Celtic','American Indian / Choctaw'],'Native_America')
    
    # Mixed Race: 5 (remember to look at this in relation to all the mixed white people/lation peop)
    df['Race'] = df['Race'].replace(['Mixed','mixed','Caucasian / Black / West Indian','Hispanic/Black','Black/White',
                                    'Mix','Black and White','white/hispanic','White/Hispanic',
                                     'Mutt','Black / Caucasian','European Mutt','White/Asian',
                                     'White & African American','Caucasian / Chinese', 'Caucasian / Philipino',
                                     'Hispanic or Latino; Mixed','Japanese/White','White/Latina','Hispanic/White',
                                     'Latino/White', 'Caucasian/Hispanic','Hispanic / Caucasian','Caucasian/Pacific Islander',
                                     'Filipino/Caucasian', 'African American/American Indian','White/Native American',
                                     'Middle Easter/White', 'European/Native American', 'Caucasian / Brazilian',
                                     'Asian / Pacific Islander / Hispanic', 'Caucasian / Hispanic', 'Caucasion/Filipino',
                                     'American Indian / Caucasian', 'Lebanese / American','Caucasian / Pacific Islander',
                                     'Navajo / Portugese','WASP / Italian'], 'Mixed_Race')
    
    # European: 6 (You cannot assume they are white because they are European)
    df['Race'] = df['Race'].replace(['German','Italian','Polish','Dutch','European','Swiss','North European',
                                    'Canadian','Finnish','german','Scottish','Ukrainian',
                                    'Romanian','Bulgarian','Spanish','Lithuanian','British or English','Scandinavian',
                                     'Western European','Irish','Scot - Irish','French-Canadian','Swiss/American',
                                     'Eastern European', 'French Acadian'], 'European_National')
    
    # Race Not on Application/ Not specified: 7 
    df['Race'] = df['Race'].replace(['Prefer not to say', 'No Application - Letter Only','Not on Application', 'not latino or hispanic',
                                     'not hispanic or lation','Not hispanic or Lation','not hispanice or latino',
                                     'Not Hispnic or Latino', 'Not hispanic or Latino', 'Not Hispanice or Latino',
                                     'Not Hispanic of Latino', 'Not Hispanic or Latino','Not hispanic or latino',
                                     'nor hispanic or latino','not hispanic or latino','Not Hispanic or Lation',
                                     'Not Hispanic or latino'], 'Unspecified')
    
    # Mixed Nationality 8
    df['Race'] = df['Race'].replace(['German/Irish American','Irish/American','Italian American','German / Slovak',
                                     'Yugoslav / Swedish','Irish / Scottish / German', 'Chinese / German',
                                     'European American','Scottish-German','English / Irish', 'French Canadian',
                                     'French Canadian/American','Irish/Spanish','Irish / Mexican', 'Appalachian / American',
                                     'Scotttish/Irish/Norwegian','Chinese/German','German / American',
                                     'Irish/Italian/English/Dutch','Japanese / Scandinavian','Irish/Italian',
                                     'Spanish/Italian','Italian / French', 'British / American','European / American',
                                     'Italian / Portuguese / American', 'Mexican/Greek', 'Irish American',
                                     'African / European / Carribean','Irish / American', 'Lebanese French Canadian',
                                     'Korean / English','Western European / American'], 'Mixed_Nationality')
    
    # Other: 9 
    df['Race'] = df['Race'].replace(['Viking','Other','Elf','Human','other','Vikng','Pennsylvania Dutch',
                                     'Texan','Yankee',
                                     'Hawaiian','Melungeon','American','Angelic','Alaska Native','Jewish',
                                     'American Mutt','Appalachian American', 'Polsih American'], 'Other')
    
    
     

clean_9(at)

In [11]:
# I break this column up into North Americans, Europeans, Africans, Australians 
def clean_10(df):
# North Americans 
    df['Country'] = df['Country'].replace(['USA','Canada','CANADA'],'North American')
#Europeans
    df['Country'] = df['Country'].replace(['England','Germany','France','HOLLAND','Austria',
                                           'GERMANY','THE NETHERLANDS','Switzerland','SWITZERLAND',
                                           'Ireland','Norway','ITALY','Finland','Sweden','Czech Republic'],'European')
# Oceania
    df['Country'] = df['Country'].replace(['Australia','AUSTRALIA','New Zealand'],'Oceania/Australia')

# African 
    df['Country'] = df['Country'].replace(['South Africa','Morocco (North Africa)'],'African')

# Other 
    df['Country'] = df['Country'].replace(['GUAM','Bahamas','India'],'Other')
    

clean_10(at)
    

In [12]:
def clean_11(df):
#US states. I was able to grab a list of states from a python list off stack overflow
    df['State'] = df['State'].replace(["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY",'Ca', 'Ga','Co','Me', ],0)
# Non US states (I used the key from Collections import Counter model)
    
    df['State'] = df['State'].replace(['En', 'Ho', 'MY', 'SK', 'Ta', 'BY', 'ON', 
                                        'AP', 'NB', 'Cz', 'Ke', 'Ir', 'Sw', 'Ne', 'So', 'Ha', 
                                        'Gu', 'Fi', 'PR', 'QC', 'Cu', 'NL', 'No', 'NP', 
                                        'Il', 'AE', 'VI', 'Qu', 'Fr', 'Mo', 'Au', 'Su', 'We', 
                                        'Ma', 'NS', 'La', 'BC', 'zz', 'AB', 'PE', 'Mi', 'Ge', 
                                        'UK', 'Pr', 'MB', 'CH'],1)
clean_11(at)

In [13]:
at.columns

Index(['Finish', 'Age', 'Race', 'State', 'Sex', 'Country', 'Hike_Type',
       'Start', 'completion_yr', 'HikedWDog', 'WinterHike', 'Under2KSpent',
       'SchoolCredit', 'Minority', 'Vegetarian', 'DehydrateOwnFood', 'Giardia',
       'Lyme', 'Norovirus', 'Total_Days'],
      dtype='object')

In [14]:
# def clean_12(df):
#     df['Hike_Type'] = df['Hike_Type'].map({'NOBO':0, 'SOBO':1, 'Flip':2 })
# clean_12(at)

In [15]:
def clean_13(df):
    df['Start'] = df['Start'].dt.month
    df['Finish'] = df['Finish'].dt.month

clean_13(at)

In [16]:
at.columns

Index(['Finish', 'Age', 'Race', 'State', 'Sex', 'Country', 'Hike_Type',
       'Start', 'completion_yr', 'HikedWDog', 'WinterHike', 'Under2KSpent',
       'SchoolCredit', 'Minority', 'Vegetarian', 'DehydrateOwnFood', 'Giardia',
       'Lyme', 'Norovirus', 'Total_Days'],
      dtype='object')

In [17]:
def clean_15(df):
    df['Total_Days']= df['Total_Days'].dt.days

clean_15(at)

In [18]:
def clean_16(x):
    if x == True:
        return False
    elif x == False:
        return True

at['Over2kSpent'] = at['Under2KSpent'].apply(clean_16)

def clean_17(x):
    if x == True:
        return False
    elif x == False:
        return True
at['SummerHike'] = at['WinterHike'].apply(clean_17)


In [19]:
def clean_18(x):
    if 5 <= x < 15:
        return '5-15'
    elif  15 <= x < 25:
        return '15-25'
    elif  25 <= x < 35:
        return '25-35'
    elif  35 <= x < 45:
        return '35-45'
    elif  45 <= x < 55:
        return '45-55'
    elif  55 <= x < 65:
        return '55-65'
    elif  65 <= x < 75:
        return '65-75'
    elif  75 <= x < 85:
        return '75-85'
    elif  x >= 85:
        return '85 and older'
at['Age'] = at['Age'].apply(clean_18)

In [20]:
dummies_sex = pd.get_dummies(at[['Sex','Country','Race','Hike_Type','Age']])
at_new = pd.concat([at.drop(['Sex','Country','Race','Hike_Type','Age'],axis =1),dummies_sex],axis =1)

In [21]:
dummies_sex.head()

,Sex_F,Sex_M,Country_African,Country_European,Country_North American,Country_Oceania/Australia,Country_Other,Race_African_American,Race_Asian,Race_Caucasian,...,Hike_Type_NOBO,Hike_Type_SOBO,Age_15-25,Age_25-35,Age_35-45,Age_45-55,Age_5-15,Age_55-65,Age_65-75,Age_75-85
8,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
10,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
20,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
36,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
def clean_20(df):
    df.reset_index(inplace = True, drop = True)
clean_20(at_new)

In [23]:
at_new.head()

,Finish,State,Start,completion_yr,HikedWDog,WinterHike,Under2KSpent,SchoolCredit,Minority,Vegetarian,...,Hike_Type_NOBO,Hike_Type_SOBO,Age_15-25,Age_25-35,Age_35-45,Age_45-55,Age_5-15,Age_55-65,Age_65-75,Age_75-85
0,12,0,1,1948,False,False,False,False,False,False,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10,0,5,1951,False,False,False,False,False,False,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,9,0,5,1951,False,False,False,False,False,False,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,9,0,5,1959,False,False,False,False,False,False,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,9,0,4,1965,False,False,False,False,False,False,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
at_new['Total_Days'].median()

173.0

In [25]:
features = [value for value in at_new.columns if value != 'Total_Days' and value != 'Start' and value != 'Finish']
X = at_new[features]
y = at_new['Total_Days']

In [26]:
from sklearn.cross_validation import train_test_split

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=2003)

In [28]:
from sklearn.linear_model import LinearRegression

In [29]:
lm = LinearRegression()

In [30]:
lm.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [31]:
# variable increases time by x amount 
results = pd.DataFrame(lm.coef_, features, columns = ['coef'] )
results


,coef
State,-0.949887
completion_yr,-3.083384
HikedWDog,14.350401
WinterHike,18.352690
Under2KSpent,-5.230520
SchoolCredit,-10.942790
Minority,5.098544
Vegetarian,-3.856365
DehydrateOwnFood,6.446468
Giardia,18.732793


In [32]:
pred = lm.predict(X_test)

In [33]:
from sklearn.metrics import r2_score

In [34]:
r2_score(y_test,pred)

0.12581394792929979

In [35]:
results.to_csv("results.csv")